In [1]:
import warnings
warnings.filterwarnings(action='ignore')
from hdbcli import dbapi
import pandas as pd
import os
from pathlib import Path
import numpy as np
import math
from datetime import date,datetime
pd.options.display.max_columns = 200
pd.set_option('display.max_rows',None)

In [2]:
def generaCosecha(fecha):
    '''
    El parámetro de entrada es una fecha (datetime), y retorna un String con formato YYYYmm, 
    útil para hacer analisis de cosechas:
    Ejemplo:
    generaCosecha(date(2024,1,23)) ------>  '202401'
    
    '''
    if not isinstance(fecha, date):
        raise ValueError("La entrada debe ser un objeto de tipo date.")
    cosecha = fecha.strftime("%Y%m")
    return cosecha

In [7]:
def obtener_rutas_usuario(base_local: Path = None, base_remoto: str = None, usar_remoto: bool = False):
    """
    Devuelve las rutas de datos para el usuario actual, usando pathlib.
    Ajusta la ruta si estás ejecutando desde 'notebooks/', considerando la ruta relativa.

    Parámetros:
    - base_local: ruta base local donde se encuentran los datos, considerando la estructura cookiecutter (por defecto: '../data/')
    - base_remoto: ruta base remota tipo GCP, cuando estas en la VM
    - usar_remoto: True para rutas de cloud (base_remoto), False para rutas locales (base_local)

    Retorna:
    - dict con rutas: rawData, externalData, interimData, processedData
    """
    if base_local is None:
        try:
            base_local = Path(__file__).resolve().parent.parent / 'data'
        except NameError:
            base_local = Path.cwd()
            if 'notebooks' in base_local.parts:
                base_local = base_local.parent
            base_local = base_local / 'data'

    if base_remoto is None:
        base_remoto = 'gs://default-bucket/ScoreCI'

    if usar_remoto:
        rutas = {
            'rawData': f'{base_remoto}/raw/',
            'externalData': f'{base_remoto}/external/',
            'interimData': f'{base_remoto}/interim/',
            'processedData': f'{base_remoto}/processed/'
        }
    else:
        rutas = {
            'rawData': base_local / 'raw',
            'externalData': base_local / 'external',
            'interimData': base_local / 'interim',
            'processedData': base_local / 'processed'
        }

    return rutas

In [8]:
rutas = obtener_rutas_usuario()

In [9]:
rutas

{'rawData': WindowsPath('D:/Users/antmiguel/Repos/ent-sandbox-fdo-jupyter-repository/ScoreCI/data/raw'),
 'externalData': WindowsPath('D:/Users/antmiguel/Repos/ent-sandbox-fdo-jupyter-repository/ScoreCI/data/external'),
 'interimData': WindowsPath('D:/Users/antmiguel/Repos/ent-sandbox-fdo-jupyter-repository/ScoreCI/data/interim'),
 'processedData': WindowsPath('D:/Users/antmiguel/Repos/ent-sandbox-fdo-jupyter-repository/ScoreCI/data/processed')}

In [3]:
usuario='UP_BA_ANTMIGUEL'
pssw='N2F1D#o0--'
con=dbapi.connect(address='DAIRA', port=30015, user= usuario, password=pssw)
cursor = con.cursor()

In [41]:
query = '''
select * from CLIENTES_CI
'''

In [42]:
df=pd.read_sql_query(query,con=con)

In [43]:
df.shape

(836194, 32)

In [44]:
df.to_parquet(os.path.join(rutas.get('rawData'),'ScoreCI_raw_BaseAnaliticaCliente_ScoreCI_02062025_abr25v6.parquet'), index=False)

In [ ]:
CLIENTES_CI:
ScoreCI_raw_BaseAnaliticaCliente_ScoreCI_02062025_abr25v6

ASESORESF:
ScoreCI_raw_BaseAnaliticaAsesor_ScoreCI_02062025_abr25v6

In [ ]:
df['CONTRACT_START_DT'] = pd.to_datetime(df['CONTRACT_START_DT'], errors='coerce')
df['POSITION_START_DT'] = pd.to_datetime(df['POSITION_START_DT'], errors='coerce')
df['EMPLOYEE_AGREEMENT_DT'] = pd.to_datetime(df['EMPLOYEE_AGREEMENT_DT'], errors='coerce')
df['EMPLOYEE_FIRST_AGREEMENT_DT'] = pd.to_datetime(df['EMPLOYEE_FIRST_AGREEMENT_DT'], errors='coerce')
df['EMPLOYEE_STATUS_START_DT'] = pd.to_datetime(df['EMPLOYEE_STATUS_START_DT'], errors='coerce')
df['CONTRACT_BAL_START_DT'] = pd.to_datetime(df['CONTRACT_BAL_START_DT'], errors='coerce')
df['CONTRACT_BAL_END_DT'] = pd.to_datetime(df['CONTRACT_BAL_END_DT'], errors='coerce')
df['CORTE'] = pd.to_datetime(df['CORTE'], errors='coerce')

In [ ]:
ls_limites = [-math.inf, 1, 8, 60, math.inf]
ls_etiquetas = ['a.CR0', 'b.CR1a7', 'd.CR8a60', 'e.CR60+']
df['BUCKET_ATRASO']  = pd.cut(df['DIAS_ATRASO_ACTUAL'], bins=ls_limites, labels=ls_etiquetas, right=False)

In [ ]:
df['COSECHA_CONTRATO'] = df['CONTRACT_START_DT'].apply(lambda row: generaCosecha(row))
df['MES_CORTE'] = df['CORTE'].apply(lambda row: generaCosecha(row))

In [ ]:
pd.pivot_table(
    data=df,
    index='COSECHA',
    columns='MES_CORTE',
    values = ['MONTO_CARTERA'],
    aggfunc={'MONTO_CARTERA':'sum'},
    fill_value=0
).reset_index().to_clipboard()

In [ ]:
df.groupby('COSECHA')['ASESOR_CORTE'].nunique()

In [ ]:
df[(df.MES_CORTE =='202412') & (df.ASESOR_CORTE==86176238)].shape

In [ ]:
pd.pivot_table(
    data=df[df.BUCKET_ATRASO == 'd.CR8a60'],
    index='COSECHA',
    columns='MES_CORTE',
    values = ['MONTO_CARTERA'],
    aggfunc={'MONTO_CARTERA':'sum'},
    fill_value=0
).reset_index().to_clipboard()

### contratos

In [ ]:
pd.pivot_table(
    data=df,
    index='COSECHA',
    columns='MES_CORTE',
    values = ['CONTRACT_ID'],
    aggfunc={'CONTRACT_ID':'count'},
    fill_value=0
).reset_index().to_clipboard()

In [ ]:
pd.pivot_table(
    data=df[df.BUCKET_ATRASO == 'd.CR8a60'],
    index='COSECHA',
    columns='MES_CORTE',
    values = ['CONTRACT_ID'],
    aggfunc={'CONTRACT_ID':'count'},
    fill_value=0
).reset_index().to_clipboard()